# Contexto ✒️

Esse notebook se refere a um estudo sobre a criação de Agents, que podem ser compreendidos como inteligências artificiais com capacidade de ação e interação com o meio, retirando ou enviando para ele informação.

Tais inteligências artificiais alteram o status quo antes vigente de seu uso passivo no qual, por exemplo, seus outputs só eram gerados mediante a presença de uma pessoa humana interagindo consigo via prompts. Agora, por outro lado, a presença desse não se faz necessária, a qual pode ser substituída por outra IA, que iterativamente realizam o processo de prompt, objetivando formar, passo a passo, um output ótimo ao contexto de sua aplicação.  

Para melhor compreensão sobre o que são Agents :
https://docs.crewai.com/core-concepts/Agents/

## Bibliotecas 📚

In [4]:
# Instalando os componentes pertinentes no notebook

!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [3]:
# Instalando o google gemini

!pip install crewai langchain_google_genai

In [5]:
import warnings
warnings.filterwarnings('ignore')

import google.generativeai as genai

import os
import crewai
import langchain

from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI


## Criando o modelo de LLM a ser utilizado 🤖



In [6]:
# Criando o modelo de llm utilizado com o google gemini.

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-pro-latest",
    temperature = 0.8,
    google_api_key="your_api_key"
)

### Criando os Agents

Como pode-se compreender por meio da literatura, os Agents são os recursos que permitem a automatização dos processos para eles passados, quando integrados com uma LLM. Nesse sentido, para o seu melhor desempenho, configura-se o seu papel (role), o seu objetivo (goal) e o seu contexto (backstory), na forma que cada qual, integrando-os, contextualiza melhor a LLM de uso, de modo que ela passa a prover respostas mais acuradas e adequadas ao que se espera.

In [7]:
# Planejador de conteúdo :

planner = Agent(
    role = "Planejador de conteúdo",
    goal = "Planeje o conteúdo envolvente e factualmente preciso sobre o {topico}",
    backstory = "Você está trabalhando no planejamento de um artigo de blog sobre o tópico : {topico}."
                "Você precisa coletar informações que ajudam a audiência aprender algo e tomar decisões informadas."
                "O seu trabalho é fundamental, a base, para o Escritor de Conteúdo um artigo sobre esse tópico.",
    allow_delegation = False,
    verbose = True,              # Para poder ver o Agent desempenhando a sua tarefa com base em sua regra, objetivo e backstory (contexto).
    llm = llm
)


In [8]:
# Escritor de conteúdo :

writer = Agent(
    role = "Escritor de conteúdo",
    goal = "Escreva um artigo perspicaz e factualmente acurado sobre o tópico : {topico}.",
    backstory = "Você está trabalhando na escrita de um artigo sobre a sua acurada opinião sobre o tópico : {topico}."
                "Você irá basear a sua escrita no trabalho realizado pelo Planejador de Conteúdo, que forneceu um destacado e relevante contexto sobre o tópico."
                "Você precisa seguir os objetivos principais e direções destacadas pelo Planejador de Conteúdo."
                "Você também pode fornecer objetivos e ideias parciais sobre o topico, apoiando-se no que fora trazido pelo Planejador de Conteúdo."
                "Você reconhece em seu artigo quando as suas opiniões são apenas opiniões, em oposição às declarações objetivas.",
    allow_delegation = False,
    verbose = True,
    llm = llm
)


In [9]:
# Editor do conteúdo :

editor = Agent(
    role = "Editor",
    goal = "Edite o artigo de blog, alinhando o estilo de escrita com a organização.",
    backstory = "Você é um editor que recebeu um artigo de blog de um Escritor de Conteúdo."
                "Seu objetivo é rever o artigo de blog, buscando garantir as seguintes melhores práticas jornalísticas :"
                "Prover pontos de vistas balanceadas, quando oferecer opiniões e afirmações."
                "Evitar grandes tópicos controvérsos ou opiniões quando possível.",
    allow_delegation = False,
    verbose = True,
    llm = llm
)

## Criando as tarefas

As tarefas se referem ao o que deseja-se que os Agents façam. Para a melhor qualidade de saída, recomenda-se que informe ao Agent uma descrição de sua atividade, passo a passo, de forma clara e corente, o que ele precisa realizar, informando em conjunto com o resultado esperado. Para a sua implementação, ver abaixo.

In [10]:
# Tarefa - planejar

plan = Task(
    description=(
        "1. Tenha como prioridade as últimas trends, jogadores chave e notícias importantes sobre o {topico}. \n"
        "2. Identifique a sua audiência alvo, considerando seus interesses e pontos de preocupação, apelo."
        "3. Desenvolva um conteúdo detalhado, destacando os pontos principais para uma introdução, pontos chave e chamada para ação (call to action)."
        "4. Inclua SEO keywords e relevantes dados ou fontes, que podem servir de repertório à criação do artigo do blog."
    ),
    expected_output = "Um compreensível documento de planejamento de conteúdo, com pontos de destaque, audiência, SEO keywords e fontes utilizadas.",
    agent = planner
)

In [11]:
# Tarefa - escrever

write = Task(
    description=(
        "1. Use o conteúdo planejado para criar um artigo de blog atrativo sobre o tópico {topico}. \n"
        "2. Incorpore SEO keywords naturalmente."
        "3. Crie Seções/Subtítulos que sejam nomeadas que sejam atrativos, buscando despertar o interesse no leitor."
        "4. Garanta que o artigo esteja estruturado com uma envolvente introdução, inpirador desenvolvimento e com uma conclusão sumarizada, que forneça uma síntese do que fora escrito."
        "5. Corrija erros gramáticos e alinhe os textos com o tom da marca."
    ),
    expected_output = "Um artigo de blog bem escrito, em formato markdown, pronto para publicação, com cada seção tendo de 2 a 3 parágrafos.",
    agent = writer
)

In [12]:
# Tarefa - editar

edit = Task(
    description=(
        "Analise o artigo de blog obtido, revisando possíveis erros gramaticais ou ortográficos e os corrija."
        "Alinhe o texto com o tom de voz da marca"),
    expected_output = "Um artigo de blog bem escrito, em formato markdown, pronto para publicação, com cada seção tendo de 2 a 3 parágrafos.",
    agent = editor
)

### Crew (Integrando os Agents com suas tarefas, produzindo um resultado final)

Essa etapa se refere à integração dos Agents com suas tarefas, em ordem de obter a saída final do que fora estipulado : um artígo atraente sobre um tópico informado.

Para isso, passa-se as tarefas em ordem cronológca, uma vez que uma influi na outra, em ordem que a saída de uma se torna a entrada de outra e assim por diante.

Para melhor compreensão ver a célula abaixo.

In [13]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose = 2 # O valor 2 indica que queremos ver a saída, em debug, completa de cada etapa da sequência : ( planner - plan ) -> ( writer - write ) -> ( editor - edit )
)

In [14]:
# "Colocando os Agents para funcionar" :

result = crew.kickoff(inputs={"topico": "Bring Me The Horizon"})


 [DEBUG]: == Working Agent: Planejador de conteúdo
 [INFO]: == Starting Task: 1. Tenha como prioridade as últimas trends, jogadores chave e notícias importantes sobre o Bring Me The Horizon. 
2. Identifique a sua audiência alvo, considerando seus interesses e pontos de preocupação, apelo.3. Desenvolva um conteúdo detalhado, destacando os pontos principais para uma introdução, pontos chave e chamada para ação (call to action).4. Inclua SEO keywords e relevantes dados ou fontes, que podem servir de repertório à criação do artigo do blog.


> Entering new CrewAgentExecutor chain...
Thought: I can definitely create a comprehensive content plan for a blog article about Bring Me The Horizon, covering their latest trends, key players, important news, and more. This will help the Content Writer craft an engaging and informative piece.

## Final Answer: 

**Blog Article Content Plan: Bring Me The Horizon**

**I.  Target Audience:**

* **Primary:** Existing fans of Bring Me The Horizon, music en

In [15]:
from IPython.display import Markdown

# Formatando a saída em Markdown, para que seja mais atraente visualmente.

Markdown(result)

## Bring Me The Horizon: A Sonic Odyssey From Screams to Synthesizers 

Bring Me The Horizon is not just a band; they are a sonic experience. From their early days as fiery deathcore prodigies to their current reign as genre-bending icons, their journey is a testament to constant evolution and creative fearlessness. This is the story of how five lads from Sheffield, UK, conquered the world, one earth-shattering riff at a time.

Formed in 2004, the band burst onto the scene with a raw, chaotic energy seldom seen in metalcore. Their debut album, *Count Your Blessings* (2006), was an explosion of ferocious breakdowns and vocalist Oli Sykes’s signature guttural screams. It was an immediate hit amongst heavy music fans, establishing Bring Me The Horizon as a force to be reckoned with.

## Evolving Soundscapes: A Journey Through Albums

However, it was their sophomore album, *Suicide Season* (2008), that truly put them on the map. The album marked a departure from their debut, showcasing a more experimental side with melodic elements woven into their signature aggressive sound. Tracks like “Chelsea Smile” and “The Sadness Will Never End” became anthems for a generation grappling with intense emotions.

Bring Me The Horizon continued to push boundaries with each subsequent release. *There Is a Hell, Believe Me I've Seen It. There Is a Heaven, Let's Keep It a Secret.* (2010) delved into darker lyrical themes and incorporated orchestral and electronic elements, hinting at sonic explorations to come. The release of *Sempiternal* in 2013 marked a turning point. With Jordan Fish joining as a full-time member, their sound became increasingly infused with electronic textures and soaring melodies. The album was a critical and commercial triumph, solidifying their status as arena-filling headliners.

Their experimentation continued with *That’s The Spirit* (2015) and *Amo* (2019). These albums saw the band embrace pop sensibilities, incorporating elements of electronic dance music, and further exploring Sykes’s impressive vocal range. While this bold evolution divided some longtime fans, it also attracted a whole new audience captivated by their willingness to defy expectations.

## Post Human: Survival Horror -  A Commentary on Modernity

In 2020, Bring Me The Horizon unveiled the first installment of their ambitious *Post Human* project. *Post Human: Survival Horror* is a concept album that grapples with the anxieties and uncertainties of the modern world.  The album blends metalcore, electronica, pop-punk, and industrial influences into a cohesive and exhilarating listening experience. 

Lyrically, *Survival Horror* is their most politically charged work to date, addressing topics like climate change, social unrest, and technology's impact on our lives.  The album features collaborations with artists like Babymetal, YUNGBLUD, and Nova Twins, further diversifying their sound and amplifying the album's message of unity and resilience. 

##  Beyond the Music: A Visual and Cultural Force

Bring Me The Horizon's artistry extends far beyond the music. Their visual identity, from their iconic album art to their elaborate stage productions, is integral to their overall impact.  Sykes, also a talented artist, has often been hands-on with their visual direction, creating a cohesive aesthetic that is instantly recognizable.

Moreover, the band's influence on the alternative music scene is undeniable. They have inspired countless young artists, not only through their music but also through their message of embracing individuality and challenging conventions. 

## Join the Conversation: What's Your Bring Me The Horizon Era?

Bring Me The Horizon's journey has been nothing short of remarkable. They've consistently pushed the boundaries of genre, inspiring fierce loyalty from fans and sparking endless debates about their artistic direction. But one thing is certain: Bring Me The Horizon refuses to be put in a box. They are a band in constant motion, forever evolving, and their story is far from over. 

What do you think of Bring Me The Horizon's musical evolution? What's your favorite album era?  Share your thoughts in the comments below!